# Goal: Create Customer Profiles for an LGS

In [1]:
import wrangle as w

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Methodology:

1. Divide store items into categories
2. Calculate the total number of items that each customer has purchased in each category
3. Use dbscan to cluster customers and create groups based on those clusters
4. Analyze each group and develop a profiles based on that analysis 

# Prepare Steps

## General

1. Read in sales data by year and merge files into one dataframe <br>
<br>
2. Remove non-payment rows <br>
<br>
3. Create datetime column and set index to the datetime column <br>
<br>
4. Delete unused columns and Rename remaining columns for ease of use <br>
<br>
5. Clean text in ID <br>
<br>
4. Clean text in net_sales and convert to int <br>
<br>
5. Clean text in cart and convert items string to a list of those items <br>
<br>
6. Fill Null values using
  * ‘unregistered’ for the id column <br>
  * ‘No-discount’ for the discount  <br>
<br>
7. Add a column for each item showing the nuber of that item bought in each transaction <br>
<br>
8. Add a column for each category of item showing the number of items bought in that catagory  <br>

## Project Specific

1. Restrict data to purchases made in 2023 <br>
<br>
2. Drop all unrelated columns <br>
<br>
3. Drop rows where id is unregistered <br>
<br>
4. Create scaled columns for each category column <br>

In [6]:
df = w.get_prepared_data()
df

,id,cart,discount,gross_sales,discount_amount,net_sales,year,month,day,weekday,...,zona:_the_secrets_of_chernobyl,accessories,board_games,concessions,modeling_supplies,role_playing_games,minis_models,trading_card_games,other,all_items
2021-01-01 15:07:02,T49C25V8WS37VB4RSJTBN13TSR,['dragon_shield_sleeves:_matte_blue'],no_discount,$11.99,$0.00,11.99,2021,1,1,Friday,...,0,1,0,0,0,0,0,0,0,1
2021-01-01 15:09:20,F36VZJMBMH3PH5SN664GPV4NQR,['ultra_pro-100_deck_box_white_2020_-_aw12892'],no_discount,$5.99,$0.00,5.99,2021,1,1,Friday,...,0,1,0,0,0,0,0,0,0,1
2021-01-01 16:30:31,BBBRQPD57S3YV4GJG1CCB6ATGC,"['candy', 'dungeons_&_dragons:_icewind_dale', ...",no_discount,$31.25,$0.00,31.25,2021,1,1,Friday,...,0,0,0,1,0,1,1,0,0,3
2021-01-01 16:31:27,W67T53BZFD5VQ8J3W7387X8SW0,"['candy', 'double_sided_battlemap_-_chx96246']",no_discount,$21.25,$0.00,21.25,2021,1,1,Friday,...,0,0,0,1,0,0,1,0,0,2
2021-01-01 19:36:24,F4RYR1ARCD7ZDD4E9PFYRVF6H0,"['dex_binder', '3_x_custom_amount', 'candy', '...",Military,$257.14,-$25.50,231.64,2021,1,1,Friday,...,0,1,0,2,0,0,0,0,4,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 16:06:59,9QF5CQDWN165DD0QQTJDESHQN0,"['gwco_ultramarines_blue_-_29-18', 'gwla_thund...",15% off entire sale (up to $15.00 off),$23.73,-$3.56,20.17,2023,12,31,Sunday,...,0,0,0,1,3,0,0,0,0,4
2023-12-31 17:07:49,unregistered,"['wilds_of_eldraine_set_booster', 'streets_of_...",no_discount,$12.98,$0.00,12.98,2023,12,31,Sunday,...,0,0,0,0,0,0,0,2,0,2
2023-12-31 17:11:52,8V1DCCMP0104HDHCTCJRKZ7HTG,"['water_bottle', 'a&w_root_beer', 'lance_toast...",10%,$11.06,-$1.11,9.95,2023,12,31,Sunday,...,0,0,0,2,0,0,1,1,0,4
2023-12-31 18:41:21,4FH2YZRSWS1P7F2Y9CP65AA37R,['2_x_spots'],no_discount,$59.98,$0.00,59.98,2023,12,31,Sunday,...,0,0,2,0,0,0,0,0,0,2


In [13]:
from sklearn.preprocessing import MinMaxScaler

# restrict data to data from 2023
df = df['2023-01-01' : '2023-12-31']
df

,id,cart,discount,gross_sales,discount_amount,net_sales,year,month,day,weekday,...,zona:_the_secrets_of_chernobyl,accessories,board_games,concessions,modeling_supplies,role_playing_games,minis_models,trading_card_games,other,all_items


In [ ]:
# revove purchases not tied to an id number
df = df[df.id != 'unregistered']

# get relevent columns
df = df[['id', 
       'accessories',
       'board_games', 
       'concessions', 
       'modeling_supplies', 
       'role_playing_games',
       'minis_models', 
       'trading_card_games',
       'net_sales']]

In [ ]:
cata_df = df.groupby('id').agg(sum)
cata_df.describe()
cata_df.info()

In [ ]:
cata_df[cata_df.trading_card_games > 1000]

In [ ]:
df.resample('M').agg(sum)

In [ ]:
 cols_to_scale = ['accessories',
                  'board_games', 
                  'concessions', 
                  'modeling_supplies', 
                  'role_playing_games',
                  'minis_models', 
                  'trading_card_games',
                  'net_sales']
    

    

In [ ]:
to_scale_df = df[cols_to_scale]

scaler = sklearn.preprocessing.MinMaxScaler().fit(train_to_be_scaled)
    
scaled_df = scaler.transform(train_to_be_scaled)

for col in cols_to_scale:
    
    scaled_df = scaled_df.rename(columns={col: col + "_scaled"})
    
df = df.join(scaled_df)

# Data Dictionary

Rows in the original data represent a record for each purchase made at LGS in the years 2021, 2022, and 2023. <br>
The final transformation of the data will contain columns showing the average number of purchases made by a given customer across the following categories in a given month. It will also show the average net sales (after removing discounts) for each customer for each month.


|Category|Description|Examples|
|--------|-----------|--------|
|Accessories|Items that enhance game play or are used to store game play items|Binders, Dice, Card Sleeves|
|Board Games|Self contained board games and board game expansions|Terraforming Mars, LOTR Journies in Middle Earth|
|Concessions|Food and drink items|drinks, candy|
|Minis/Models|Miniature models, contained in customizable table top minis games or sold as stand alone minis or sets of minis Does not include boardgames that contain minis|Warhammer Minis, D&D Minis|
|Modeling Supplies|Items used to enhance appearance of minis/models|Painting Supplies, Model Bases|
|Role Playing Games|Books and map packs for Role Playing Games|Dungeons and Dragons Books, Pathfinder Books|
|Trading Card Games|Cards for customizable card games|Magic, Pokemon, Yugio|

In [ ]:
df.head()

In [ ]:
from itertools import combinations


cols = ['accessories', 
        'board_games', 
        'concessions',
        'modeling_supplies', 
        'role_playing_games', 
        'minis_models',
        'trading_card_games']

combs = combinations(cols, 2)

for pair in combs:
    
    plt.scatter(df[f'{pair[0]}'], df[f'{pair[1]}'])
    
    plt.xlable(f'{pair[0]}')
    plt.ylable(f'{pair[1]}')
    
    plt.show()